<a href="https://colab.research.google.com/github/Ahmed9588406/Chess/blob/master/FaceID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import uuid
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import patoolib
# metrics calculation to calculate Precision and Recall
from matplotlib.pyplot import figure
from tensorflow.keras.metrics import Precision, Recall
import cv2 as cv
"""
 Using Functional API allows us to be specific in our creation of a NN
 as it can accept multiple input and out note stack(linear) like sequential model
"""
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input, Layer,Dropout 
from tensorflow.keras.losses import BinaryCrossentropy

"""
# Model : to compile all the layers together
# Layer : is a class which allows us to create a specific layer in our Network
# Dense : fully connected layer
# MaxPooling : allows us to shrink the information which we got A technic to
reduce information in an image while minting features
Input : another base class to input the image 
# Flatten : take the output from the Previous layer an make it in lone dimension
"""


'\n# Model : to compile all the layers together\n# Layer : is a class which allows us to create a specific layer in our Network\n# Dense : fully connected layer\n# MaxPooling : allows us to shrink the information which we got A technic to\nreduce information in an image while minting features\nInput : another base class to input the image \n# Flatten : take the output from the Previous layer an make it in lone dimension\n'

In [ ]:
! pip install patool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')  # getting the different gpus on our machine
for gpu in gpus:  # looping throw them
    tf.config.experimental.set_memory_growth(gpu, True)  # to set our gpus to true


In [ ]:
for gpu in gpus:
    print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
patoolib.extract_archive("data.zip")

patool: Extracting data.zip ...
patool: running /usr/bin/7z x -o./Unpack_gr4fswm5 -- data.zip
patool: ... data.zip extracted to `data'.


'data'

In [ ]:
# Setup our paths
 
POS_PATH = os.path.join("data", "positive")
NEG_PATH = os.path.join("data", "negative")
ANC_PATH = os.path.join("data", "anchor")

# Establish a connection to the Camera

In [ ]:
"""
Establish a connection to the Camera
"""


cap = cv.VideoCapture(0)
while cap.isOpened():
    # read(): return two values one as a boolean and a frame from the camera
    ret, frame = cap.read()

    # cut down frame to 250x250px
    frame = frame[120:120 + 250, 200:200 + 250, :]

    # Collect anchors
    if cv.waitKey(1) & 0XFF == ord("a"):
        imgname = os.path.join(ANC_PATH, f"{uuid.uuid1()}.jpg")
        cv.imwrite(imgname, frame)

    # Collecting positive
    if cv.waitKey(1) & 0XFF == ord("p"):
        imgname = os.path.join(POS_PATH, f"{uuid.uuid1()}.jpg")
        cv.imwrite(imgname, frame)

    # Now we will show the the video frame by frame
    cv.imshow("Image Collection", frame)

    # waitKey(1) will display a frame for 1 ms, after which display will be automatically closed.
    if cv.waitKey(1) & 0XFF == ord('q'):
        break

# Release the camera
cap.release()

# Close the image show frame
cv.destroyAllWindows()


# Data Augmentation


In [ ]:
# Data Augmentation 

def data_aug(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img,min_jpeg_quality=90,max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9, upper=1, seed=(np.random.randint(100), np.random.randint(100)))
        data.append(img)
    return data                                                

# Testing 1


In [ ]:
# Testing 1
img_path = os.path.join(ANC_PATH, "4c3bd0a4-c001-11ed-a93f-ec2e980717f3.jpg")
img = cv.imread(img_path)
augmented_imgs = data_aug(img)

for image in augmented_imgs:
    cv.imwrite(os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1())), image.numpy())

# Testing 2 


In [ ]:
# Testing 2 
for file_name in os.listdir(os.path.join(POS_PATH)):
    img_path = os.path.join(POS_PATH, file_name)
    img = cv.imread(img_path)
    augmented_imgs = data_aug(img)
    
    for image in augmented_imgs:
        cv.imwrite(os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1())), image.numpy())

# Get Images Directories


In [ ]:
import tensorflow as tf
import os

# Set the directory paths where the image files are stored
ANC_PATH = 'data/anchor/'
POS_PATH = 'data/positive/'

# Define a function to load image files from a directory
def load_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image

# Load image files from the anchor directory
anchor_ds = tf.data.Dataset.list_files(os.path.join(ANC_PATH, '*.jpg'))
anchor_ds = anchor_ds.map(load_image)

# Load image files from the positive directory
positive_ds = tf.data.Dataset.list_files(os.path.join(POS_PATH, '*.jpg'))
positive_ds = positive_ds.map(load_image)

# Print the number of images in each dataset
print('Number of anchor images:', tf.data.experimental.cardinality(anchor_ds).numpy())
print('Number of positive images:', tf.data.experimental.cardinality(positive_ds).numpy())

Number of anchor images: 9649
Number of positive images: 5590


In [ ]:
import os
import shutil

# Set the directory paths where the image files are stored
ANC_PATH = 'data/anchor/'
POS_PATH = 'data/positive/'

# Get a list of all the image files in the positive directory
image_files = os.listdir(POS_PATH)

# Copy the first 10 images from the positive directory to the anchor directory
for i in range(400):
    image_file = image_files[i]
    source_path = os.path.join(POS_PATH, image_file)
    destination_path = os.path.join(ANC_PATH, image_file)
    shutil.copyfile(source_path, destination_path)

In [ ]:
import shutil

# Set the directory path to delete
dir_path = 'data'

# Delete the directory and all its contents
shutil.rmtree(dir_path)

In [ ]:
import os

# Set the directory path where the image files are stored
POS_PATH = 'data/positive/'

# Get a list of all the image files in the positive directory
image_files = os.listdir(POS_PATH)

# Print the number of image files in the positive directory
print('Number of image files:', len(image_files))

Number of image files: 5590


In [ ]:
 anchor = tf.data.Dataset.list_files(ANC_PATH + '\*.jpg').take(4000)
      
positive = tf.data.Dataset.list_files(POS_PATH + '\*.jpg').take(4000)
negative = tf.data.Dataset.list_files(NEG_PATH + '\*.jpg').take(4000)

InvalidArgumentError: ignored

In [ ]:
import tensorflow as tf

ANC_PATH = 'data/anchor/*.jpg'
POS_PATH = 'data/positive/*.jpg'
NEG_PATH = 'data/negative/*.jpg'

anchor_files = tf.data.Dataset.list_files(ANC_PATH).take(4000)
positive_files = tf.data.Dataset.list_files(POS_PATH).take(4000)
negative_files = tf.data.Dataset.list_files(NEG_PATH).take(4000)

print(f"Number of anchor files: {len(list(anchor_files))}")
print(f"Number of positive files: {len(list(positive_files))}")
print(f"Number of negative files: {len(list(negative_files))}")


Number of anchor files: 4000
Number of positive files: 4000
Number of negative files: 4000


In [ ]:
def preprocess(file_path):
    # Read image from file_path ( read the file as a byte object)"""
    # A bytes object is an immutable sequence of bytes, conceptually similar to a string. of a bytes object
    # is an unsigned int that satisfies 0 b 1111 _ 1111 >= x >= 0.
    # """
    byte_img = tf.io.read_file(file_path)
    # Load in the image
    img = tf.io.decode_jpeg(byte_img)
    # Preprocessing steps - resizing the image to be 100x100x3 channels (as it's a RGP image)
    img = tf.image.resize(img, (100, 100))
    # Scale the image to be between 0 and 1
    # Every single pixel value traditionally from 0 to 255 to be 0 to 1
    # that will help us in Gradient Descent
    img = img / 255.0
    # Return the image
    return img

In [ ]:
# Allow us to iterate throw them all
"""
(anchor, positive) -> 1, 1, 1, 1, 1
(anchor, negative) -> 0, 0, 0, 0, 0
"""
# Create our labels
# hint tf.ones (len(anchor) will return a numpy array with the shape of 300(100x100x3) and dtype=float32
positives = tf.data.Dataset.zip((anchor_files, positive_files, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor_files)))))
# Separate dataset for the negative
negatives = tf.data.Dataset.zip((anchor_files, negative_files, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor_files)))))
# the shape of data is ((), (), (), (tf.string , tf.string, tf.float32))
# Make the dataset of the positive and negative together in one big dataset
data = positives.concatenate(negatives)

In [ ]:
# this function will take two images and preprocessing them
def preprocess_twin(input_image, validation_image, label):
    return preprocess(input_image), preprocess(validation_image), label

In [ ]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)

In [ ]:
"""
data shape will be : ((100, 100, None), (100, 100, None), ()), types( float32, float32, float32)
"""

# Training partition
# Here will take 70% of our data (500 * 70% = 350) as we can pass 350 directly
train_data = data.take(round(len(data) * .7))
# we will not input the images one by one we will input them by batches (16)
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

# data shapes : ((None, 100, 100, None), (None, 100, 100, None), (None,)), types :(tf.float32, tf.float32, tf.float32)

# Testing partition
test_data = data.skip(round(len(data) * 0.7))
test_data = test_data.take(round(len(data) * 0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [ ]:
# Making the Layers
"""def make_Embedding():
    inp = Input(shape=(100, 100, 3), name="input_image")

    # First block of the network
    conv1 = Conv2D(64, (10, 10), activation='relu')(inp)
    maxpooling1 = MaxPool2D(64, (2, 2), padding="same")(conv1)
    droup_out1 = Dropout(0.3)(maxpooling1)

    # Second block
    conv2 = Conv2D(128, (7, 7), activation="relu")(maxpooling1)
    maxpooling2 = MaxPool2D(64, (2, 2), padding="same")(conv2)
    droup_out2 = Dropout(0.3)(maxpooling2)

    # Third block
    conv3 = Conv2D(128, (4, 4), activation="relu")(maxpooling2)
    maxpooling3 = MaxPool2D(64, (2, 2), padding="same")(conv3)
    droup_out3 = Dropout(0.3)(maxpooling3)

    # Final embadding block
    conv4 = Conv2D(265, (4, 4), activation="relu")(droup_out3)
    flatten = Flatten()(conv4)
    dense1 = Dense(4096, activation="relu")(flatten)
    dense2 = Dense(1024, activation="sigmoid")(dense1)
    return Model(inputs=[inp], outputs=[dense2], name="Embedding")
"""

In [ ]:
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.regularizers import l2

def make_Embedding(input_shape):
    """
    Base network to be shared (eq. to feature extraction).
    """
    input = Input(shape=input_shape)
    x = layers.Conv2D(64, (10, 10), activation='relu', input_shape=input_shape,
                      kernel_regularizer=l2(2e-4))(input)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (7, 7), activation='relu',
                      kernel_regularizer=l2(2e-4))(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (4, 4), activation='relu', kernel_regularizer=l2(2e-4))(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, (4, 4), activation='relu', kernel_regularizer=l2(2e-4))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(4096, activation='sigmoid', kernel_regularizer=l2(1e-3))(x)

    return Model(input, x)

In [ ]:
embedding = make_Embedding((100, 100))
embedding.summary()

ValueError: ignored

In [ ]:
embedding = make_Embedding()


In [ ]:
# Siamese L1 Distance class
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()

    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)


In [ ]:
def make_siamese_model():
    # Anchor image input in Neural Network
    input_image = Input(name='input_image', shape=(100, 100, 3))

    # Validation image in Neural Network
    validation_image = Input(name="validation", shape=(100, 100, 3))

    siamese_layer = L1Dist()
    siamese_layer._name = "distance"
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    # Classification
    classifier = Dense(1, activation='sigmoid')(distances)
    return Model(inputs=[input_image, validation_image], outputs=classifier, name="SiameseNetwork")


In [ ]:
siamese_network = make_siamese_model()
siamese_network.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation (InputLayer)        [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 Embedding (Functional)         (None, 1024)         44501321    ['input_image[0][0]',            
                                                                  'validation[0][0]']

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(1e-4)  

In [ ]:
"""
Note : we can load the wights of this network for training another project by
model.load('path_to_checkpoint) this will load the pretrained weights into the existing model
"""
chekpoint_directory = './training_checkpoints'
# we will have our consistent format of ckpt
chekpoint_prefix = os.path.join(chekpoint_directory, 'ckpt')

checkpoint = tf.train.Checkpoint(opt=optimizer, siamese_network=siamese_network)

In [ ]:
"""
@tf.function : Compiles a function into a callable TensorFlow graph
"""


# Build Train step function
@tf.function
def train(batch):
    # that's allow us to record and calculate the gradient descent (Backpropagation)
    with tf.GradientTape() as tape:
        # Get anchor and positive / negative image
        x = batch[:2]
        y = batch[2]
        # Forward pass
        # setting the training in our model to true will activate the layers to be trained ( like dropout)
        yhat = siamese_network(x, training=True)
        # Calculate Loss
        loss = binary_cross_loss(y, yhat)

        # Calculate gradients(Gradient Descent)
        grad = tape.gradient(loss, siamese_network.trainable_variables)
        # Calculate update weights and apply to siamese network
        # The optimizer here is calculating and propagating the new weights using Adam's optimisation algorithm
        optimizer.apply_gradients(zip(grad, siamese_network.trainable_variables))
        # Return Loss
        return loss

In [ ]:
def train_loop(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS + 1):
        print(f"\n Epoch {epoch}/{EPOCHS}")
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object
        r = Recall()
        p = Precision()

        # Loop through each batch
        for index, batch in enumerate(data):
            
            
            # Run train function here
            loss = train(batch)
            yhat = siamese_network.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat)
            progbar.update(index + 1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())   

        # Save the checkpoints every 10 epoch
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=chekpoint_prefix)


In [ ]:
Epochs = 50

train_loop(train_data, Epochs)



Streaming output truncated to the last 5000 lines.
350/350 [==============================] - 121s 345ms/step
0.076222904 0.99144995 0.9953505

 Epoch 6/50
350/350 [==============================] - 122s 349ms/step
0.13235003 0.99462944 0.9953422

 Epoch 7/50
350/350 [==============================] - 121s 345ms/step
0.038907044 0.9946865 0.9911754

 Epoch 8/50
350/350 [==============================] - 121s 345ms/step
0.061597507 0.99749017 0.9928622

 Epoch 9/50
350/350 [==============================] - 121s 346ms/step
0.010745142 0.99542737 0.99612814

 Epoch 10/50
350/350 [==============================] - 120s 341ms/step
0.017048392 0.99752825 0.9929701

 Epoch 11/50
350/350 [==============================] - 122s 348ms/step
0.03635474 0.9960145 0.99493307

 Epoch 12/50
350/350 [==============================] - 125s 356ms/step
0.04507739 0.9989158 0.997114

 Epoch 13/50
350/350 [==============================] - 121s 346ms/step
0.045719333 0.99502665 0.99502665

 Epoch 14/50
350

KeyboardInterrupt: ignored

In [ ]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_network.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true, yhat)
print( r.result().numpy(), p.result().numpy())    

1/1 [==============================] - 0s 35ms/step
1.0 0.99172187


In [ ]:
# Make prediction
y_hat = siamese_network.predict([test_input, test_val])

[1 if prediction > 0.5 else 0 for prediction in y_hat]

1/1 [==============================] - 0s 21ms/step


[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1]

In [ ]:
"""
The subplot function allows us to render plots using a grid like format
The first parameter is the number of rows and the second one is the number of columns and 
the third one is the index    
"""
# Visualization the results
plt.figure(figsize=(18, 8))
plt.subplot(1, 2, 1)
# As we know we can change the numbers that are passing in test_input or test_val
plt.imshow(test_input[0])
# As we know we can change the numbers that are passing in test_input or test_val
plt.subplot(1, 2, 2)
plt.imshow(test_val[0])
plt.show()

In [ ]:
# Save the wights of our siamese_network
siamese_network.save("Siamese_Model.h5")

In [ ]:
"""
The load_weights() method can be used to load the weights also.The save_weights() method saves only
the weights, the save() method saves optimizers as well 
"""
# Reload model
# The second line we define our custom_objects so when we are building the kivy app we will need a reference on it
model = tf.keras.models.load_model("Siamese_Model.h5",
                                   custom_objects={'L1Dist': L1Dist,
                                                   "BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy})


In [ ]:
# Making prediction with our loaded model
model.predict([test_input, test_val])
model.summary() # we should see the summary and every layer in siamese_Network

1/1 [==============================] - 0s 166ms/step
Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_image (InputLayer)       [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 validation (InputLayer)        [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 Embedding (Functional)         (None, 1024)         44501321    ['input_image[0][0]',            
                                

In [ ]:
# Making verification function
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [ ]:
# Varification
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv.imshow('Verification', frame)
    
    # Verification trigger
    if cv.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(model, 0.9, 0.7)
        print(verified)
    
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

In [ ]:
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv.imshow('Verification', frame)
    
    # Verification trigger
    if cv.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv.imwrite(os.path.join('/content/drive/MyDrive/application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(model, 0.9, 0.7)
        print(verified)
    
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

In [ ]:
import cv2 as cv
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define path to save input image
input_path = os.path.join('/content/drive/MyDrive/application_data', 'input_image', 'input_image.jpg')

# Open webcam
cap = cv.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()
    
    # Check if frame is valid
    if not ret:
        print("Error: Failed to capture frame from camera!")
        break
    
    # Crop frame to region of interest
    frame = frame[120:120+250,200:200+250, :]
    
    # Display frame
    cv.imshow('Verification', frame)
    
    # Wait for 'v' key to trigger verification process
    if cv.waitKey(10) & 0xFF == ord('v'):
        # Save input image to Google Drive
        cv.imwrite(input_path, frame)
        
        # Run verification
        results, verified = verify(model, 0.9, 0.7)
        print(verified)
    
    # Wait for 'q' key to exit program
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

# Release webcam and close windows
cap.release()
cv.destroyAllWindows()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error: Failed to capture frame from camera!


In [ ]:
import cv2 as cv
from google.colab.patches import cv2_imshow

cap = cv.VideoCapture(0)

while True:
    # Read frame from webcam
    ret, frame = cap.read()

    # Check if frame is valid
    if not ret:
        print("Error: Failed to capture frame from camera!")
        break

    # Display frame
    cv2_imshow(frame)

    # Wait for 'q' key to exit program
    if cv.waitKey(1) == ord('q'):
        break

# Release webcam and close windows
cap.release()
cv.destroyAllWindows()